In [17]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.graph.message import add_messages

from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import Annotated, List
from typing_extensions import TypedDict
import os
load_dotenv()
google_api_key=os.getenv('google_api_key')

In [18]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model=GEMINI_MODEL, temperature=0.3)

In [19]:
class State(TypedDict):
    messages: Annotated[List, add_messages]



In [20]:
from composio_langgraph import Action, ComposioToolSet, App
composio_toolset = ComposioToolSet(api_key="ui9jbd366fmtu1y6ekotd")
tools = composio_toolset.get_tools(apps=[App.GOOGLECALENDAR])

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


In [21]:
llm_with_tools = llm.bind_tools(tools)


In [22]:
graph_builder = StateGraph(State)


In [23]:
tool_node = ToolNode(tools=tools)


In [24]:
def chatbot(state: State):
    """ travel assistant that answers user questions about their trip.
    Depending on the request, leverage which tools to use if necessary."""
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

In [25]:
graph_builder.add_node("chatbot", chatbot)

        
graph_builder.add_node("tools", tool_node)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.set_entry_point("chatbot")

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
memory=MemorySaver()
graph=graph_builder.compile(checkpointer=memory)

In [26]:
def chat(input:str):
    config = {"configurable": {"thread_id": "1"}}
    response=graph.invoke({'messages':HumanMessage(content=str(input))},config)
    return response['messages'][-1].content

In [41]:
chat('list events starting with today')

'OK. I have retrieved the events in your primary calendar starting from today. They are: Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday, Loanne birthday.'